In [1]:
# Libraries
import math, re, os
from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets
import json
import shutil
import zipfile
import re
import tensorflow as tf

# EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from joblib import Parallel, delayed

# Data
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

E0000 00:00:1724792686.581981      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0827 21:04:46.590296170      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0827 21:04:46.590311413      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0827 21:04:46.590314907      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0827 21:04:46.590317623      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0827 21:04:46.590320191      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

Tensorflow version 2.16.1


In [2]:
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1724792701.262521      13 service.cc:145] XLA service 0x565b664c5350 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724792701.262577      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1724792701.262582      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1724792701.262585      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1724792701.262588      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1724792701.262591      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1724792701.262593      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1724792701.262596      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1724792701.262599      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [3]:
!pip install --upgrade kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.6 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=867d02a97ed28de7606da0e298faa196088458ccb426ff0d37ca89c764ccf38e
  Stored in directory: /root/.cache/pip/wheels/9f/af/22/bf406f913dc7506a485e60dce8143741abd0a92a19337d83a3
Successfully built kaggle

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# Kaggle API Using Secrets

from kaggle_secrets import UserSecretsClient

# Brought key info
user_secrets = UserSecretsClient()
kaggle_key = user_secrets.get_secret("jiyoungroh")

# Setting
os.environ['KAGGLE_KEY'] = kaggle_key
os.environ['KAGGLE_USERNAME'] = "jiyoungroh"

In [5]:
!kaggle competitions download -c playground-series-s4e8

 98%|█████████████████████████████████████▍| 81.0M/82.3M [00:02<00:00, 44.3MB/s]
100%|██████████████████████████████████████| 82.3M/82.3M [00:02<00:00, 37.5MB/s]


In [6]:
# Unzip
with zipfile.ZipFile('/kaggle/working/playground-series-s4e8.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

# Loading Data
train_data = pd.read_csv('/kaggle/working/train.csv')
test_data = pd.read_csv('/kaggle/working/test.csv')
sample_submission = pd.read_csv('/kaggle/working/sample_submission.csv')

In [7]:
pd.set_option('display.max_columns', None)
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


# 1. Data Preprocessing

1. 수치형 변수들 로그화 (나중에 성능 안 좋으면 기존 변수로 사용해보기)
2. Stem-width와 Cap-diameter 처리 (PCA) or 모델 학습할 때 정규화나 교차 검증해서 과적합 방지하기 (PCA 먼저 해볼거임)
3. 범주형 변수들 random forest나 lasso, chi-square 등 사용해서 feature selection 할 예정
4. 모델링 한 다음에 성능 보고 위에 과정들 중 선택해서 다시 해보기


## 1. lOG

In [8]:
# Train Feature Setting

train_numeric_columns =  train_data[['cap-diameter', 'stem-height', 'stem-width']]

train_categoric_columns = train_data[['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root',
    'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']]                                 

train_remove_categoric_columns = train_data[[
    'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color',
    'stem-surface', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season'
]]
                                

In [9]:
# Test Feature Setting

test_numeric_columns =  test_data[['cap-diameter', 'stem-height', 'stem-width']]

test_categoric_columns = test_data[['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root',
    'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']]                                 

test_remove_categoric_columns = test_data[[
    'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color',
    'stem-surface', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season'
]]
       

In [10]:
# Train Log Numeric
train_data['log_cap-diameter'] = np.log1p(train_data['cap-diameter'])
train_data['log_stem-height'] = np.log1p(train_data['stem-height'])
train_data['log_stem-width'] = np.log1p(train_data['stem-width'])

In [11]:
# Test Log Numeric
test_data['log_cap-diameter'] = np.log1p(test_data['cap-diameter'])
test_data['log_stem-height'] = np.log1p(test_data['stem-height'])
test_data['log_stem-width'] = np.log1p(test_data['stem-width'])

## 2. Remove or Replace Missing Value

### 1. Numeric Data

Remove or Replace Missing Value in Numeric (average or mean..)

Remove - cap-diameter, stem-width

Replace - Median(before log), Mean(after log) - stem-height


In [12]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,log_cap-diameter,log_stem-height,log_stem-width
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,2.282382,1.706565,2.796671
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,1.706565,1.756132,2.012233
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,2.071913,2.060514,2.391511
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,1.585145,1.640937,2.018895
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,1.924249,1.474763,2.236445


In [13]:
# Remove numeric NA from train data
train_data = train_data.dropna(subset=['cap-diameter', 'stem-width'])

In [14]:
# Replace stem-height NA to median value
train_data['stem-height'].fillna(train_data['stem-height'].median(), inplace=True)

In [15]:
# Replace log stem-height NA to mean value
train_data['log_stem-height'].fillna(train_data['log_stem-height'].mean(), inplace=True)

## 2. Categoric Data

Remove NaN: cap-shape, cap-color, does-bruise-or-bleed, gil-color, stem-color, has-ring, habitat, season

KNN Imputation: cap-surface, gill-attachment, gill-spacing, stem-surface, ring-type

Remove Column: stem-root, veil-type, veil-color, spore-print-color

In [16]:
# Remove Na in train data 
train_data = train_data.dropna(subset=['cap-shape', 'cap-color', 'does-bruise-or-bleed', 
                                       'gill-color', 'stem-color', 'has-ring', 
                                       'habitat', 'season'])

In [17]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,log_cap-diameter,log_stem-height,log_stem-width
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,2.282382,1.706565,2.796671
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,1.706565,1.756132,2.012233
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,2.071913,2.060514,2.391511
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,1.585145,1.640937,2.018895
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,1.924249,1.474763,2.236445


In [18]:
test_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,log_cap-diameter,log_stem-height,log_stem-width
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a,2.265921,2.495682,2.897016
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a,2.066863,0.819780,2.463853
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,3.14,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s,1.098612,1.971299,1.420696
3,3116948,3.47,x,t,n,f,s,c,n,4.98,8.51,NaN,NaN,w,NaN,n,t,z,NaN,d,u,1.497388,1.788421,2.252344
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,13.70,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u,1.969906,2.045109,2.687847


In [19]:
# Train Label Encoder
for col in ['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']:
    le = LabelEncoder()
    combined_data = pd.concat([train_data[col]], axis=0).astype(str)
    le.fit(combined_data)
    train_data[col] = le.transform(train_data[col].astype(str))

In [20]:
# Test Label Encoder
for col in ['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']:
    le = LabelEncoder()
    combined_data = pd.concat([test_data[col]], axis=0).astype(str)
    le.fit(combined_data)
    test_data[col] = le.transform(test_data[col].astype(str))

In [21]:
#Train

# KNN Imputer reset(k=5 is basic)
imputer = KNNImputer(n_neighbors=5)

# KNN Imputation run (cap-surface, gill-attachment, gill-spacing, stem-surface, ring-type)
train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']] = imputer.fit_transform(
    train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']]
)

In [22]:
# Test

# KNN Imputer reset(k=5 is basic)
imputer = KNNImputer(n_neighbors=5)

# KNN Imputation run (cap-surface, gill-attachment, gill-spacing, stem-surface, ring-type)
train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']] = imputer.fit_transform(
    train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']]
)

In [23]:
# Remove Column (stem-root, veil-type, veil-color, spore-print-color)
train_data = train_data.drop(columns=['stem-root', 'veil-type', 'veil-color', 'spore-print-color'])
test_data = test_data.drop(columns=['stem-root', 'veil-type', 'veil-color', 'spore-print-color'])

In [24]:
print(train_data.isna().sum())

id                      0
class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-surface            0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
log_cap-diameter        0
log_stem-height         0
log_stem-width          0
dtype: int64


In [25]:
print(test_data.isna().sum())

id                       0
cap-diameter             7
cap-shape               31
cap-surface              0
cap-color               13
does-bruise-or-bleed    10
gill-attachment          0
gill-spacing             0
gill-color              49
stem-height              1
stem-width               0
stem-surface             0
stem-color              21
has-ring                19
ring-type                0
habitat                 25
season                   0
log_cap-diameter         7
log_stem-height          1
log_stem-width           0
dtype: int64


## 2. PCA

In [26]:
# Train

# Data Prepration for PCA
PCA_numeric_data = train_data[['cap-diameter', 'stem-width']].values
PCA_log_numeric_data = train_data[['log_cap-diameter', 'log_stem-width']].values

# PCA Model
pca = PCA(n_components=1)  # Decrease 1 components
pca_result = pca.fit_transform(PCA_numeric_data)

# Add Result of PCA to new column
train_data['pca_cap_stem'] = pca_result

In [27]:
#Test

# Data Prepration for PCA
PCA_numeric_data = test_data[['cap-diameter', 'stem-width']].values
PCA_log_numeric_data = test_data[['log_cap-diameter', 'log_stem-width']].values

# Impute missing values
imputer = SimpleImputer(strategy='mean')
PCA_numeric_data_imputed = imputer.fit_transform(PCA_numeric_data)
PCA_log_numeric_data_imputed = imputer.fit_transform(PCA_log_numeric_data)

# PCA Model on original and log-transformed data
pca = PCA(n_components=1)
pca_result_numeric = pca.fit_transform(PCA_numeric_data_imputed)
pca_result_log_numeric = pca.fit_transform(PCA_log_numeric_data_imputed)

# Add Result of PCA to new columns
test_data['pca_cap_stem'] = pca_result_numeric
test_data['pca_log_cap_stem'] = pca_result_log_numeric


In [28]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem
0,0,e,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435
1,1,p,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392
2,2,e,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813
3,3,e,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063
4,4,e,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960


In [29]:
test_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,pca_log_cap_stem
0,3116945,8.64,x,47,n,t,54,28,w,11.13,17.12,42,w,t,15,d,a,2.265921,2.495682,2.897016,6.389057,0.766412
1,3116946,6.90,o,54,o,f,54,17,y,1.27,10.75,42,n,f,14,d,a,2.066863,0.819780,2.463853,-0.096032,0.300806
2,3116947,2.00,b,38,n,f,54,17,n,6.18,3.14,42,n,f,14,d,s,1.098612,1.971299,1.420696,-9.086434,-1.115188
3,3116948,3.47,x,54,n,f,58,17,n,4.98,8.51,42,w,t,36,d,u,1.497388,1.788421,2.252344,-3.618022,-0.211247
4,3116949,6.17,x,39,y,f,56,28,y,6.73,13.70,42,y,t,23,d,u,1.969906,2.045109,2.687847,2.230671,0.421076


In [30]:
len(test_data)

2077964

## 3. Feature Selection

1. Filter Method
2. Wrapper Method
3. Embedded Method

In [31]:
# Filter Method
selected_features = [
    'cap-surface', 'gill-color', 'stem-color', 'habitat','stem-surface', 'pca_cap_stem']

In [32]:
def chunked_one_hot_encoding(data, column, chunk_size=50000):
    num_chunks = len(data) // chunk_size + 1
    encoder = OneHotEncoder(sparse_output=False, dtype=int)
    
    # 구간별로 인코딩된 데이터 저장
    encoded_data = []
    
    for i in range(num_chunks):
        chunk = data.iloc[i*chunk_size:(i+1)*chunk_size]
        encoded_chunk = encoder.fit_transform(chunk[[column]])
        encoded_df = pd.DataFrame(encoded_chunk, columns=[f"{column}_{cat}" for cat in encoder.categories_[0]])
        encoded_data.append(encoded_df.reset_index(drop=True))
    
    # 구간별로 인코딩된 데이터를 합침
    return pd.concat(encoded_data, axis=0).reset_index(drop=True)

# class 변수에 대해 원 핫 인코딩 수행
encoded_class = chunked_one_hot_encoding(train_data, 'class')

# 원래 데이터에서 class 열을 제거하고 인코딩된 class 열을 추가
train_data = train_data.drop(columns=['class']).reset_index(drop=True)
train_data = pd.concat([train_data, encoded_class], axis=1)

In [33]:
train_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,class_e,class_p
0,0,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435,1,0
1,1,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392,0,1
2,2,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813,1,0
3,3,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063,1,0
4,4,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960,1,0


In [34]:
def chunked_target_encoding(train_data, test_data, target, col, chunk_size=50000, n_splits=5):
    num_chunks = len(train_data) // chunk_size + 1
    mean_target = train_data[target].mean()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    train_encoded_full = np.zeros(train_data.shape[0])
    test_encoded_full = np.zeros(test_data.shape[0])

    for i in range(num_chunks):
        train_chunk = train_data.iloc[i*chunk_size:(i+1)*chunk_size].copy()
        test_chunk = test_data.copy()

        train_encoded = np.zeros(train_chunk.shape[0])
        test_encoded = np.zeros(test_chunk.shape[0])

        for train_index, val_index in kf.split(train_chunk):
            train_fold, val_fold = train_chunk.iloc[train_index], train_chunk.iloc[val_index]
            means = train_fold.groupby(col)[target].mean()
            
            train_encoded[val_index] = val_fold[col].map(means).fillna(mean_target)
            test_encoded += test_chunk[col].map(means).fillna(mean_target) / n_splits

        train_encoded_full[i*chunk_size:(i+1)*chunk_size] = train_encoded
        test_encoded_full += test_encoded / num_chunks

    return train_encoded_full, test_encoded_full

# 타겟 인코딩 적용
categorical_features = ['cap-surface', 'gill-color', 'stem-color', 'habitat', 'stem-surface']
for col in categorical_features:
    train_data[col + '_encoded'], test_data[col + '_encoded'] = chunked_target_encoding(train_data, test_data, 'class_e', col)

# 인코딩된 변수들만 사용하도록 업데이트
encoded_features = [col + '_encoded' for col in categorical_features] + ['pca_cap_stem']

In [35]:
train_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,class_e,class_p,cap-surface_encoded,gill-color_encoded,stem-color_encoded,habitat_encoded,stem-surface_encoded
0,0,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435,1,0,0.632735,0.565330,0.576750,0.470406,0.485669
1,1,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392,0,1,0.529051,0.284326,0.399862,0.470406,0.304646
2,2,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813,1,0,0.625532,0.565320,0.391055,0.610882,0.604030
3,3,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063,1,0,0.530814,0.507954,0.572575,0.466357,0.483924
4,4,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960,1,0,0.559783,0.561877,0.573767,0.330876,0.483230


In [36]:
test_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,pca_log_cap_stem,cap-surface_encoded,gill-color_encoded,stem-color_encoded,habitat_encoded,stem-surface_encoded
0,3116945,8.64,x,47,n,t,54,28,w,11.13,17.12,42,w,t,15,d,a,2.265921,2.495682,2.897016,6.389057,0.766412,0.412177,0.571461,0.582284,0.471075,0.44711
1,3116946,6.90,o,54,o,f,54,17,y,1.27,10.75,42,n,f,14,d,a,2.066863,0.819780,2.463853,-0.096032,0.300806,0.433243,0.420886,0.388992,0.471075,0.44711
2,3116947,2.00,b,38,n,f,54,17,n,6.18,3.14,42,n,f,14,d,s,1.098612,1.971299,1.420696,-9.086434,-1.115188,0.459809,0.285116,0.388992,0.471075,0.44711
3,3116948,3.47,x,54,n,f,58,17,n,4.98,8.51,42,w,t,36,d,u,1.497388,1.788421,2.252344,-3.618022,-0.211247,0.433243,0.285116,0.582284,0.471075,0.44711
4,3116949,6.17,x,39,y,f,56,28,y,6.73,13.70,42,y,t,23,d,u,1.969906,2.045109,2.687847,2.230671,0.421076,0.459809,0.420886,0.306154,0.471075,0.44711


In [37]:
selected_features = [
   'id', 'cap-surface_encoded', 'gill-color_encoded', 'stem-color_encoded', 'habitat_encoded','stem-surface_encoded', 'pca_cap_stem']

In [38]:
# 80:20 비율로 train과 test 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(train_data[selected_features], train_data[['class_e', 'class_p']], test_size=0.2, random_state=42)

# 4. Modeling

## 4-1. Basic Modeling (Logistic Regression, Random Forest, XGBoost)

### 1. Logistic Regression

with strategy.scope():
    model_lr = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model_lr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history_lr = model_lr.fit(X_train, y_train['class_e'], epochs=10, batch_size=1024, validation_split=0.2)

### 2. Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_rf = RandomForestClassifier(warm_start=True, random_state=42)
n_trees = 10  # Total number of trees
step = 1  # Number of trees to add at each iteration
accuracies = []

for i in range(step, n_trees + step, step):
    model_rf.set_params(n_estimators=i)
    model_rf.fit(X_train, y_train['class_e'])

    preds_rf = model_rf.predict(X_train)
    acc_rf = accuracy_score(y_train['class_e'], preds_rf)
    accuracies.append(acc_rf)
    print(f"Number of Trees: {i}, Accuracy: {acc_rf:.4f}")

# Final model fit
model_rf.fit(X_train, y_train['class_e'])

### 3. XG Boost

!pip install xgboost

import xgboost as xgb

model_xgb = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False)
model_xgb.fit(X_train, y_train['class_e'])

### Save Model

import joblib

# TPU에서 CPU로 모델 복사
with tf.device('/cpu:0'):
    model_lr_cpu = tf.keras.models.clone_model(model_lr)
    model_lr_cpu.set_weights(model_lr.get_weights())

# 모델 저장
model_lr_cpu.save('/kaggle/working/Logistic_regression_model.keras')
joblib.dump(model_rf, '/kaggle/working/random_forest_model.pkl')
joblib.dump(model_xgb, '/kaggle/working/XG_Boost_model.pkl')

In [44]:
import joblib

# Specify the full path to the model file
model_path = '/kaggle/input/random_forest/scikitlearn/default/1/random_forest_model (1).pkl'

# Load the model
model_rf = joblib.load(model_path)

print("Model loaded successfully.")

Model loaded successfully.


In [45]:
X_train.head()

,id,cap-surface_encoded,gill-color_encoded,stem-color_encoded,habitat_encoded,stem-surface_encoded,pca_cap_stem
1156571,1156660,0.619996,0.577433,0.588074,0.470289,0.489813,7.684469
1232942,1233037,0.545909,0.574131,0.388914,0.474284,0.308248,-1.580802
720878,720936,0.088862,0.280113,0.394678,0.468210,0.488246,-10.586996
2792302,2792508,0.360854,0.421347,0.312170,0.472976,0.493214,3.643701
149767,149771,0.355245,0.419360,0.298458,0.473682,0.486559,5.101259


In [46]:
test_data = test_data[selected_features]

In [47]:
test_data.head()

,id,cap-surface_encoded,gill-color_encoded,stem-color_encoded,habitat_encoded,stem-surface_encoded,pca_cap_stem
0,3116945,0.412177,0.571461,0.582284,0.471075,0.44711,6.389057
1,3116946,0.433243,0.420886,0.388992,0.471075,0.44711,-0.096032
2,3116947,0.459809,0.285116,0.388992,0.471075,0.44711,-9.086434
3,3116948,0.433243,0.285116,0.582284,0.471075,0.44711,-3.618022
4,3116949,0.459809,0.420886,0.306154,0.471075,0.44711,2.230671


In [79]:
preds_rf_test = model_rf.predict(test_data)

In [80]:
# Ensure preds_rf_test is in the correct format (list or array of integers)
preds_rf_test = preds_rf_test.astype(int)

# Convert predictions: 0 -> 'e', 1 -> 'p'
class_predictions = ['e' if pred == 0 else 'p' for pred in preds_rf_test]

# Create a DataFrame with the IDs and the converted predictions
results_df = pd.DataFrame({
    'id': test_data['id'],
    'class': class_predictions
})

# Display the first 10 rows
print(results_df.head(10))

        id class
0  3116945     e
1  3116946     e
2  3116947     e
3  3116948     p
4  3116949     e
5  3116950     e
6  3116951     e
7  3116952     p
8  3116953     e
9  3116954     p


In [81]:
len(results_df)

2077964

In [82]:
# Save the DataFrame to a CSV file
results_df.to_csv('/kaggle/working/results_rf_test.csv', index=False)

# Confirm the file has been saved
print("File saved as: /kaggle/working/results_rf_test.csv")

File saved as: /kaggle/working/results_rf_test.csv


## 4-2. Random Forest Hyper Parameter Tuning